In [ ]:
from azureml.core import Dataset, Datastore
from azureml.data.datapath import DataPath
from azureml.core.workspace import Workspace
from azureml.core.authentication import ServicePrincipalAuthentication

sp = ServicePrincipalAuthentication(tenant_id="", # tenantID
                                    service_principal_id="", # clientId
                                    service_principal_password="") # clientSecret


ws = Workspace.get(subscription_id="6c304488-2903-485c-a8bd-404858da811b",
                   resource_group="AzureML",
                   name="PipelineML",
                   auth=sp)


# create tabular dataset from a SQL database in datastore
datastore = Datastore.get(ws, 'sqbconnector')
query = DataPath(datastore, 'SELECT TradeDate, BCHprice, FinalSentiment FROM Finaldata')
tabular = Dataset.Tabular.from_sql_query(query, query_timeout=10)

In [ ]:
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core import Workspace
from azureml.core.model import Model
from azureml.core.conda_dependencies import CondaDependencies
import joblib
from azureml.automl.core.forecasting_parameters import ForecastingParameters

forecasting_parameters = ForecastingParameters(time_column_name='TradeDate', 
                                               forecast_horizon=12,
                                               freq='D',
                                               target_lags='auto')


train_data = tabular
label = "BCHprice"



cluster = ws.compute_targets['crypto-ml-cluster']

from azureml.core.experiment import Experiment
from azureml.train.automl import AutoMLConfig
import logging
from azureml.core.runconfig import RunConfiguration

automl_config = AutoMLConfig(task='forecasting',
                             primary_metric='normalized_root_mean_squared_error',
                             experiment_timeout_minutes=15,
                             compute_target = cluster,
                             enable_early_stopping=True,
                             training_data=train_data,
                             label_column_name=label,
                             n_cross_validations=5,
                             enable_ensembling=False,
                             verbosity=logging.INFO,
                             forecasting_parameters=forecasting_parameters)



In [ ]:
import sklearn

experiment = Experiment(ws, "Mlfcstm")
remote_run = experiment.submit(automl_config, show_output=True)


In [ ]:
import mlflow
import pandas as pd
import datetime
today = pd.Timestamp.today()
d = datetime.timedelta(days = 14)
fcstrange = today + d
#print(yesterday)

best_run, fitted_model = remote_run.get_output()

import numpy as np
label_fcst, data_trans = fitted_model.forecast(
     forecast_destination=fcstrange)

In [ ]:
#check how automl processed features
#fitted_model.named_steps['timeseriestransformer'].get_featurization_summary()

In [ ]:
fcstprice = list(label_fcst)
fcstdate = pd.date_range(start=today,end=fcstrange)


df = pd.DataFrame(list(zip(fcstdate, fcstprice)), 
               columns =['ForecastDate', 'ForecastPrice'])


In [ ]:
import pyodbc
server = 'bchtradingserver.database.windows.net' 
database = 'BCH_trading_data' 
username = '' 
password = ''  

driver = [item for item in pyodbc.drivers()][-1]

con_string = f'DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password}'
cnxn = pyodbc.connect(con_string)

cursor = cnxn.cursor()

#Since our table is used only for reporting needs, we don't need to keep history data
cursor.execute("TRUNCATE TABLE [dbo].[ForecastPrice]")

for index, row in df.iterrows():
    cursor.execute("INSERT INTO [dbo].[ForecastPrice] (FcstBCHprice,FcstDate) values(?,?)", row.ForecastPrice, row.ForecastDate)

cnxn.commit()
cursor.close()